## 拉钩Python招聘信息爬取

In [12]:
import random
import time

import requests
from openpyxl import Workbook
import pymysql.cursors


def get_conn():
   '''建立数据库连接'''
   conn = pymysql.connect(host='localhost',
                               user='root',
                               password='root',
                               db='python',
                               charset='utf8mb4',
                               cursorclass=pymysql.cursors.DictCursor)
   return conn


def insert(conn, info):
   '''数据写入数据库'''
   with conn.cursor() as cursor:
       sql = "INSERT INTO `python` (`shortname`, `fullname`, `industryfield`, `companySize`, `salary`, `city`, `education`) VALUES (%s, %s, %s, %s, %s, %s, %s)"
       cursor.execute(sql, info)
   conn.commit()


def get_json(url, page, lang_name):
   '''返回当前页面的信息列表'''
   headers = {
        'Accept': 'application/json, text/javascript, */*; q=0.01',
        'Accept-Encoding': 'gzip, deflate, br',
        'Accept-Language': 'zh-CN,zh;q=0.9,und;q=0.8',
        'Connection': 'keep-alive',
        'Content-Length': '55',
        'Content-Type': 'application/x-www-form-urlencoded; charset=UTF-8',
        'Cookie': 'JSESSIONID=ABAAABAAAGFABEF2EEBD44BEFA69338D3F3F38BC09E46D3; _ga=GA1.2.1080077220.1560156899; _gid=GA1.2.1375169045.1560156899; Hm_lvt_4233e74dff0ae5bd0a3d81c6ccf756e6=1560156899; user_trace_token=20190610165458-6cf29f86-8b5d-11e9-a22c-5254005c3644; LGSID=20190610165458-6cf2a168-8b5d-11e9-a22c-5254005c3644; LGUID=20190610165458-6cf2a2f1-8b5d-11e9-a22c-5254005c3644; index_location_city=%E5%85%A8%E5%9B%BD; TG-TRACK-CODE=index_search; X_MIDDLE_TOKEN=90cb717a148ddf9b3845f2a37d2c80c3; _gat=1; Hm_lpvt_4233e74dff0ae5bd0a3d81c6ccf756e6=1560158765; LGRID=20190610172604-c5209a60-8b61-11e9-aab1-525400f775ce; X_HTTP_TOKEN=c7bd48f9315c1a2b058851065112898731c9d54a5f; SEARCH_ID=01f04a82304d48cf96899b5645af9ce2',
        'Host': 'www.lagou.com',
        'Origin': 'https://www.lagou.com',
        'Referer': 'https://www.lagou.com/jobs/list_python?city=%E5%8C%97%E4%BA%AC&cl=false&fromSearch=true&labelWords=&suginput=',
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.169 Safari/537.36',
        'X-Anit-Forge-Code': '0',
        'X-Anit-Forge-Token': 'None',
        'X-Requested-With': 'XMLHttpRequest'
   }
   data = {'first': 'false', 'pn': page, 'kd': lang_name}
   json = requests.post(url, data, headers=headers).json()
   list_con = json['content']['positionResult']['result']
   info_list = []
   for i in list_con:
       info = []
       info.append(i.get('city', '无'))
       info.append(i.get('companyShortName', '无'))  # 公司名
       info.append(i.get('companyFullName', '无'))
       info.append(i.get('industryField', '无'))   # 行业领域
       info.append(i.get('companySize', '无'))  # 公司规模
       info.append(i.get('financeStage', '无'))
       info.append(i.get('salary', '无'))   # 薪资
       info.append(i.get('workYear', '无'))   
       info.append(i.get('education', '无'))   # 学历
       info.append(i.get('createTime', '无'))  
       info_list.append(info)
   return info_list   # 返回列表


def main():
   lang_name = '后期制作'
   wb = Workbook()  # 打开 excel 工作簿
#    conn = get_conn()  # 建立数据库连接  不存数据库 注释此行
   for i in ['杭州']:   # 五个城市  爬多了会被封  先爬一个
       page = 1
       ws1 = wb.active
       ws1.title = lang_name
       url = 'https://www.lagou.com/jobs/positionAjax.json?gj=3%E5%B9%B4%E5%8F%8A%E4%BB%A5%E4%B8%8B&px=default&city={}&needAddtionalResult=false'.format(i)
       while page < 3:   # 每个城市30页信息  爬多了会被封  先爬一页
           info = get_json(url, page, lang_name)
           page += 1
           time.sleep(random.randint(30, 50))
           for row in info:
#                insert(conn, tuple(row))  # 插入数据库，若不想存入 注释此行
               ws1.append(row)
#    conn.close()  # 关闭数据库连接，不存数据库 注释此行
   wb.save('{}职位信息.xlsx'.format(lang_name))

if __name__ == '__main__':
   main()